In [ ]:
import cv2
import numpy as np
from owslib.wms import WebMapService
from pyproj import Transformer
from PIL import Image
from io import BytesIO
import requests
from osgeo import gdal

# 📍 Koordinaten und Einstellungen
lat, lon = 47.61601, 9.05841
wms_url = "https://wms.geo.admin.ch/?VERSION=1.3.0"
layer = "ch.kantone.cadastralwebmap-farbe"  # realer Layername
srs = "EPSG:3857"
size = (1500, 1500)
output_bounds_meters = 300  # WMS-Ausschnitt Breite/Höhe in Metern

# 🔁 Transformiere Koordinaten
transformer = Transformer.from_crs("EPSG:4326", srs, always_xy=True)
x_center, y_center = transformer.transform(lon, lat)

xmin = x_center - output_bounds_meters / 2
xmax = x_center + output_bounds_meters / 2
ymin = y_center - output_bounds_meters / 2
ymax = y_center + output_bounds_meters / 2

# 🗺️ WMS holen
wms = WebMapService(wms_url)
img_wms = wms.getmap(
    layers=[layer],
    styles=[""],
    srs=srs,
    bbox=(xmin, ymin, xmax, ymax),
    size=size,
    format="image/png",
    transparent=False
)
img_ref = Image.open(BytesIO(img_wms.read()))
img_ref.save("refdata/referenz.png")

# 🔍 Lade das PDF-Bild (zuvor exportiert als PNG)
img_target = cv2.imread("refdata/Bildschirmfoto 2025-05-22 um 12.05.01.png", cv2.IMREAD_GRAYSCALE)
img_ref_gray = cv2.cvtColor(np.array(img_ref), cv2.COLOR_RGB2GRAY)

# 🔑 SIFT Feature-Matching
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img_target, None)
kp2, des2 = sift.detectAndCompute(img_ref_gray, None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)

if len(good) < 10:
    raise Exception("Nicht genug gute Matches für automatische Georeferenzierung")

src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

# Homographie
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
warped = cv2.warpPerspective(cv2.imread("refdata/Bildschirmfoto 2025-05-22 um 12.05.01.png"), M, size)

cv2.imwrite("refdata/georef_image.png", warped)

# 🧭 Georeferenzierte GeoTIFF erzeugen
gdal.Translate(
    "refdata/georef_output.tif",
    "refdata/georef_image.png",
    outputSRS=srs,
    outputBounds=[xmin, ymax, xmax, ymin]  # Achtung: Y-Achse!
)

print("✅ Georeferenzierung abgeschlossen: 'refdata/georef_output.tif'")


✅ Georeferenzierung abgeschlossen: 'georef_output.tif'
